In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from tools import CALENDAR_ID, reserva_restaurante_tool,update_reservation_tool
import datetime as dt
import json
from config import TZ
from calendar_client import get_calendar_service
from zoneinfo import ZoneInfo
from db import SessionLocal, create_reservation, get_last_event_id_by_thread
# Normalizar “hoy/mañana/sábado” → YYYY-MM-DD
from dateutil.relativedelta import relativedelta, MO, TU, WE, TH, FR, SA, SU
import re
from typing import Optional


In [5]:
from menu_index import load_menu_vector
from graph import build_app, GlobalState
from db import init_db, SessionLocal,save_message, load_history
from langchain.agents import AgentExecutor, create_react_agent

init_db()
db = SessionLocal()

# Cargar el vector store del menú
vector_store = load_menu_vector(r"C:\Users\cabal\Desktop\startup\Selling_Agent\menu_casona_completo.json")

# Construir el grafo/agent
app = build_app(vector_store)

In [23]:
def get_last_event_id_by_thread(
    session,
    thread_id: str,
    require_confirmed: bool = True
) -> Optional[str]:
    """Devuelve el event_id más reciente (por updated_at/created_at) para el thread dado.
       Prioriza reservas con status='confirmed' y event_id no-nulo.
       Si no hay confirmadas y require_confirmed=False, toma la más reciente con event_id."""
    base_q = session.query(Reservation.event_id).filter(
        Reservation.thread_id == thread_id,
        Reservation.event_id.isnot(None)
    )

    if require_confirmed:
        q = base_q.filter(Reservation.status == "confirmed")
        rec = q.order_by(desc(Reservation.updated_at), desc(Reservation.created_at)).first()
        if rec and rec[0]:
            return rec[0]

    # Fallback: cualquiera con event_id
    rec = base_q.order_by(desc(Reservation.updated_at), desc(Reservation.created_at)).first()
    return rec[0] if rec and rec[0] else None